In [1]:
import cgi
import os
import requests


# Need to get a new Developer Token  before running chef because expires after one hour
BOXAPI_DEVELOPER_TOKEN = 'bHuTyXztm3nETINEM3CbhssV80HsJO3C'


BOXAPI_SHARED_ITEMS = "https://api.box.com/2.0/shared_items?fields=type,id"
BOXAPI_FILES_CONTENT = "https://api.box.com/2.0/files/{file_id}/content"
BOXAPI_FOLDER_DETAILS = 'https://api.box.com/2.0/folders/{folder_id}'
BOXAPI_FOLDER_ITEMS = 'https://api.box.com/2.0/folders/{folder_id}/items'



CHEFDATA_DIR = 'chefdata'



In [2]:
def get_shared_item(shared_link):
    headers = {
        "Authorization": "Bearer " + BOXAPI_DEVELOPER_TOKEN,
        "BoxApi": "shared_link=" + shared_link,
    }
    # GET1: get file id for this shared link
    response1 = requests.get(BOXAPI_SHARED_ITEMS, headers=headers)
    json_data = response1.json()
    shared_type, shared_id = json_data['type'], json_data['id']
    if shared_type == 'file':
        folder_id = None
        file_id = shared_id
    elif shared_type == 'folder':
        folder_id = shared_id
        file_id = None
    # print(shared_type, folder_id, file_id)
    return shared_type, folder_id, file_id



In [3]:
# file shared link
SHARE_LINK_URL = 'https://rescue.box.com/s/n3y9ofkeslvjhw3h5s8hwmb49bttnpss'
get_shared_item(SHARE_LINK_URL)

('file', None, '92007689777')

In [4]:
FOLDER_SHARE_URL = 'https://rescue.app.box.com/s/j1cua6q1d6b0ibppirh6qrkx1c4jdlmu'
get_shared_item(FOLDER_SHARE_URL)

('folder', '47664910783', None)

In [5]:
def box_download_file(file_id, shared_link, destdir=CHEFDATA_DIR):
    headers = {
        "Authorization": "Bearer " + BOXAPI_DEVELOPER_TOKEN,
        "BoxApi": "shared_link=" + shared_link,
    }
    # GET2: get actual file data
    response = requests.get(BOXAPI_FILES_CONTENT.format(file_id=file_id), headers=headers)
    content_disposition = response.headers['Content-Disposition']
    _, params = cgi.parse_header(response.headers['Content-Disposition'])
    filename = params['filename']
    out_path = os.path.join(destdir, filename)
    with open(out_path, 'wb') as outf:
        outf.write(response.content)
        print('Saved file', out_path, 'of size', len(response.content)/1024/1024, 'MB')
    return out_path


In [6]:
FOLDER_SHARE_URL = 'https://rescue.app.box.com/s/j1cua6q1d6b0ibppirh6qrkx1c4jdlmu'
box_download_file('282067463136', FOLDER_SHARE_URL, destdir=CHEFDATA_DIR)

Saved file chefdata/Resources & Handouts for Parenting Curriculum Children 6-11 IRC Greece.pdf of size 1.6020088195800781 MB


'chefdata/Resources & Handouts for Parenting Curriculum Children 6-11 IRC Greece.pdf'

In [7]:
FOLDER_SHARE_URL = 'https://rescue.app.box.com/s/j1cua6q1d6b0ibppirh6qrkx1c4jdlmu'
headers = {
    "Authorization": "Bearer " + BOXAPI_DEVELOPER_TOKEN,
    "BoxApi": "shared_link=" + FOLDER_SHARE_URL,
}

# GET1: get file id for this shared link
response1 = requests.get(BOXAPI_SHARED_ITEMS, headers=headers)
data1 = response1.json()
shared_type, shared_id = data1['type'], data1['id']
print(shared_type, shared_id)



folder 47664910783


In [8]:
def box_download_folder(folder_id, shared_link, destdir=CHEFDATA_DIR):
    """
    Return a dict {'title': '',  'children': [ {'path':'local/path/to/file.pdf'}]  }
    """
    headers = {
        "Authorization": "Bearer " + BOXAPI_DEVELOPER_TOKEN,
        "BoxApi": "shared_link=" + shared_link,
    }
    
    # Get deets
    response1 = requests.get(BOXAPI_FOLDER_DETAILS.format(folder_id=folder_id), headers=headers)
    folder_data = response1.json()
    folder_name = folder_data['name']
    folder_dict = dict(
        title=folder_name,
        children=[]
    )
    folder_path = os.path.join(CHEFDATA_DIR, folder_name)
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)

    # Get contents
    response2 = requests.get(BOXAPI_FOLDER_ITEMS.format(folder_id=folder_id), headers=headers)
    json_data = response2.json()
    for entry in json_data['entries']:
        if entry['type'] == 'file':
            filename = entry['name']
            file_id = entry['id']
            file_path = box_download_file(file_id, shared_link, destdir=folder_path)
            filename = os.path.basename(file_path)
            file_dict = dict(
                title=filename,
                path=file_path
            )
            folder_dict['children'].append(file_dict)
        else:
            print('Skipping entry', entry)
    
    return folder_dict




In [9]:
box_download_folder('47664910783', FOLDER_SHARE_URL)

Saved file chefdata/Condensed Version_Children/Customized Parenting Skills Curriculum 6-11 IRC Greece IRC - FINAL VERSION.pdf of size 1.1756696701049805 MB
Saved file chefdata/Condensed Version_Children/Resources & Handouts for Parenting Curriculum Children 6-11 IRC Greece.pdf of size 1.6020088195800781 MB


{'title': 'Condensed Version_Children',
 'children': [{'title': 'Customized Parenting Skills Curriculum 6-11 IRC Greece IRC - FINAL VERSION.pdf',
   'path': 'chefdata/Condensed Version_Children/Customized Parenting Skills Curriculum 6-11 IRC Greece IRC - FINAL VERSION.pdf'},
  {'title': 'Resources & Handouts for Parenting Curriculum Children 6-11 IRC Greece.pdf',
   'path': 'chefdata/Condensed Version_Children/Resources & Handouts for Parenting Curriculum Children 6-11 IRC Greece.pdf'}]}